# Data Collection
This notebook is dedicated to the collection and insertion of data into a MySQL database hosted on Google Cloud. The collection code will ultimately be transferred to a python module file that will be saved in the same subfolder as this document.

In [1]:
import requests
import time
import config
import json
import itertools
%load_ext autoreload
%autoreload 2

## Google Maps Directions API

API documentation available [here](https://developers.google.com/maps/documentation/directions/start?hl=en_US).

Python client for maps API available [here](https://github.com/googlemaps/google-maps-services-python)

In [21]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=config.api_key)

# Geocoding my address
geocode_my_result = gmaps.geocode(config.my_address)

# Store just latitude and longitude
my_geometry = geocode_my_result[0]['geometry']['location']

# Geocoding my girlfriend's address
geocode_gf_result = gmaps.geocode(config.gf_address)

# Store just latitude and longitude
gf_geometry = geocode_gf_result[0]['geometry']['location']

In [146]:
piopio_geometry = gmaps.geocode('pio pio northern blvd')[0]['geometry']['location']
print(piopio_geometry)

{'lat': 40.7555707, 'lng': -73.883179}


In [147]:
piopio_trip = GDirections(gf_geometry, piopio_geometry)

In [148]:
piopio_trip.get_trip_duration()

'40 minutes'

In [151]:
lines = piopio_trip.get_written_instructions()
for line in lines:
    print(line)

['Walk to Forest Hills - 71 Av', 'Subway towards World Trade Center  |  Take the E train\n', 'Walk to Northern Blvd/54 St', 'Bus towards FLUSHING MAIN ST STA via NORTHERN BL  |  Take the Q66 train\n', 'Walk to 84-02 Northern Blvd, Jackson Heights, NY 11372, USA']


TypeError: 'NoneType' object is not iterable

In [128]:
class GDirections:
    """
    A class to keep track of and display to the user only necessary parts of the Google maps searches.
    """
    
    def __init__(self, start_geometry, end_geometry):
        self._directions_json = self._retrieve_google_directions_json(start_geometry, end_geometry)
        self._trip_duration = self._calculate_trip_duration()
        
    def _retrieve_google_directions_json(self,start_geometry, end_geometry):
        
        """
        Call Google Maps API to retrieve a json with directions information
        """
        
        # Call API to 
        now = datetime.now()
        directions = gmaps.directions(
            origin = start_geometry,
            destination = end_geometry,
            mode = 'transit',
            departure_time = now
        )
        return directions
    
    def _calculate_trip_duration(self):
        
        """
        Calculate and return trip duration in minutes.
        
        Returns
        -------
        
        Trip duration as a string in minutes.
        
        e.g. '50 minutes'
        """
        
        # Store arrival time and convert to datetime
        arrival_time = self._directions_json[0]['legs'][0]['arrival_time']['text']
        arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')

        # Store arrival time and convert to datetime
        departure_time = self._directions_json[0]['legs'][0]['departure_time']['text']
        departure_datetime = datetime.strptime(departure_time, '%H:%M%p')

        # Calculate difference in seconds, convert to minutes
        trip_duration = int( (arrival_datetime - departure_datetime).seconds/60 )
        
        return str(trip_duration) + ' minutes'
    
    def get_trip_duration(self):
        
        """
        Getter method for trip_duration
        """
        
        return self._trip_duration
    
    def get_written_instructions(self):
        """
        Displays written details for the trip
        """
        result = []
        trips = self._directions_json[0]['legs'][0]['steps']
        for trip in trips:
            line = ''
            if trip.get('transit_details'):
                line = '  |  Take the ' + trip['transit_details']['line']['short_name'] + ' \n'
            result.append(trip['html_instructions'] + line)
        return print(result)

In [136]:
class test:
    """
    A class to keep track of and display to the user only necessary parts of the Google maps searches.
    """
    
    def __init__(self, directions_json):
        self._directions_json = directions_json
        self._trip_duration = self._calculate_trip_duration()
        
    def _retrieve_google_directions_json(self,start_geometry, end_geometry):
        
        """
        Call Google Maps API to retrieve a json with directions information
        """
        
        # Call API to 
        now = datetime.now()
        directions = gmaps.directions(
            origin = start_geometry,
            destination = end_geometry,
            mode = 'transit',
            departure_time = now
        )
        return directions
    
    def _calculate_trip_duration(self):
        
        """
        Calculate and return trip duration in minutes.
        
        Returns
        -------
        
        Trip duration as a string in minutes.
        
        e.g. '50 minutes'
        """
        
        # Store arrival time and convert to datetime
        arrival_time = self._directions_json[0]['legs'][0]['arrival_time']['text']
        arrival_datetime = datetime.strptime(arrival_time, '%H:%M%p')

        # Store arrival time and convert to datetime
        departure_time = self._directions_json[0]['legs'][0]['departure_time']['text']
        departure_datetime = datetime.strptime(departure_time, '%H:%M%p')

        # Calculate difference in seconds, convert to minutes
        trip_duration = int( (arrival_datetime - departure_datetime).seconds/60 )
        
        return str(trip_duration) + ' minutes'
    
    def get_trip_duration(self):
        
        """
        Getter method for trip_duration
        """
        
        return self._trip_duration
    
    def get_written_instructions(self):
        """
        Displays written details for the trip
        """
        trips = self._directions_json[0]['legs'][0]['steps']
        for trip in trips:
            line = ''
            if trip.get('transit_details'):
                line = '  |  Take the ' + trip['transit_details']['line']['short_name'] + ' '
            print(trip['html_instructions'], line)

In [129]:
current_trip = GDirections(my_geometry, gf_geometry)

In [139]:
test_ = test(current_trip._directions_json)

In [131]:
# Need to fix this bug where it won't display print items
current_trip.get_written_instructions()

['Walk to Grand St/Graham Av', 'Bus towards QUEENS BLVD 62 DRIVE via GRAND  |  Take the Q59 train\n', 'Walk to Grand Av - Newtown', 'Subway towards Forest Hills - 71 Av  |  Take the R train\n', 'Walk to 110-48 72nd Ave, Flushing, NY 11375, USA']


In [141]:
test_.get_trip_duration()

'50 minutes'

In [140]:
test_.get_written_instructions()

Walk to Grand St/Graham Av 
Bus towards QUEENS BLVD 62 DRIVE via GRAND   |  Take the Q59 train
Walk to Grand Av - Newtown 
Subway towards Forest Hills - 71 Av   |  Take the R train
Walk to 110-48 72nd Ave, Flushing, NY 11375, USA 
